# **IA Responsable (Responsible AI)**
**Partie 1 : Analyse de l'outil COMPAS par ProPublica**

**Partie 2 : Détection et atténuation des biais à l'aide de Fairlearn**


<img src="https://drive.google.com/uc?export=view&id=1W3nZF2AUsSbIKo_ekazO_rYDcPQSRzi8" width="50%" />

<a href="https://colab.research.google.com/drive/1tWWxMkonHYbZi_J7wDTr2Zl2CCHKyZPD" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

© Deep Learning Indaba 2024. Apache License 2.0.

**Auteurs :** Umang Bhatt et Kendall Brogle

**Introduction :**

Ce bloc-notes propose une exploration pratique de l'IA responsable en deux parties : analyser l'analyse de ProPublica de l'outil d'évaluation des risques COMPAS et examiner les biais à l'aide de la boîte à outils Fairlearn. La première partie se concentre sur l'enquête de ProPublica sur COMPAS, en particulier sur la manière dont ses scores de récidive varient selon la race et le sexe. Cela implique l'importation de données, le prétraitement, l'analyse exploratoire et la modélisation par régression logistique pour reproduire et interpréter les résultats de ProPublica. La deuxième partie passe à la détection et à l'atténuation des biais à l'aide de Fairlearn, une bibliothèque conçue pour évaluer et améliorer l'équité des modèles de Machine Learning. En abordant les aspects à la fois théoriques et pratiques de l'IA responsable, ce bloc-notes vise à améliorer la compréhension des biais dans les systèmes d'IA et des outils disponibles pour y remédier.

**Sujets :**

Contenu : IA responsable, Détection et atténuation des biais, Régression logistique, Équité dans les modèles d'IA

Niveau : Intermédiaire, Avancé


**Objectifs/Objectifs d'apprentissage :**

1) Comprendre et analyser les biais dans l'outil d'évaluation des risques COMPAS.

2) Appliquer la régression logistique pour explorer les biais raciaux dans les scores de risque.

3) Utiliser le package Fairlearn pour détecter et atténuer les biais dans les modèles.

4) Évaluer l'équité à l'aide de mesures telles que la différence et le ratio de parité démographique.


**Prérequis :**

Compréhension de base des concepts de Machine Learning.

Familiarité avec Python et des bibliothèques telles que Pandas, Scikit-learn et Matplotlib.

Compréhension de la régression logistique et des mesures de classification.

Familiarité avec les concepts de biais et d'équité en IA.

**Plan :**

>[Installation et importations](#scrollTo=6EqhIg1odqg0)

>[Partie 1 : Analyse de l'outil COMPAS par ProPublica](#scrollTo=G2sewZEq36T0)

>[Partie 2 : Détection et atténuation des biais à l'aide de Fairlearn](#scrollTo=253jTpcO60Mf)

>[Conclusion](#scrollTo=fV3YG7QOZD-B)

**Avant de commencer :**

Assurez-vous que tous les packages Python requis sont installés.

Familiarisez-vous avec l'ensemble de données et les descriptions des variables.

Revoyez les concepts clés de la régression logistique et des biais en IA.
    


## Installation et importations
    



In [ ]:
## Installation et importations requises. Capture cache la sortie de la cellule.
# @title Installer et importer les paquets requis. (Exécuter la cellule)

import subprocess
import os

# Fonction pour vérifier la présence d'un GPU/TPU et configurer l'environnement
def check_accelerator():
    try:
        subprocess.check_output('nvidia-smi')
        print("Un GPU est connecté.")
    except Exception:
        # TPU ou CPU
        if "COLAB_TPU_ADDR" in os.environ and os.environ["COLAB_TPU_ADDR"]:
            print("Un TPU est connecté.")
            import jax.tools.colab_tpu
            jax.tools.colab_tpu.setup_tpu()
        else:
            print("Seul l'accélérateur CPU est connecté.")
            # x8 périphériques CPU - nombre de périphériques hôtes (émulés)
            os.environ["XLA_FLAGS"] = "--xla_force_host_platform_device_count=8"

check_accelerator()

import jax
import jax.numpy as jnp
from jax import grad, jit, vmap

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import matplotlib.ticker as mtick
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as skm
from sklearn.metrics import accuracy_score
from IPython.display import clear_output
import math

%matplotlib inline

# Importer les données
url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"

# Pour la partie 2
# @markdown
!pip install fairlearn folktables
!git clone https://github.com/lurosenb/superquail

from folktables import ACSDataSource, ACSEmployment, ACSIncome, ACSPublicCoverage, ACSTravelTime
from superquail.data.acs_helper import ACSData
from fairlearn.datasets import fetch_adult
from fairlearn.preprocessing import CorrelationRemover
from fairlearn.reductions import ExponentiatedGradient, GridSearch, DemographicParity, ErrorRate
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.metrics import MetricFrame, demographic_parity_difference, demographic_parity_ratio, selection_rate, false_negative_rate, false_positive_rate
from IPython.display import clear_output




# **Partie 1 : Analyse de l'outil COMPAS par ProPublica**


En 2016, [ProPublica a publié une analyse](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing) de l'outil d'évaluation des risques de récidive Correctional Offender Management Profiling for Alternative Sanctions (COMPAS). COMPAS est un outil propriétaire qui génère une soi-disant évaluation des risques pour les accusés dans un procès pénal. L'analyse de ProPublica s'est concentrée sur le « score de récidive », qui est censé fournir la probabilité de récidive (c'est-à-dire de commettre un délit ou un crime) dans les deux ans suivant l'évaluation.

Dans ce laboratoire, nous allons passer en revue certaines parties de l'analyse de COMPAS par ProPublica, en nous concentrant sur la façon dont l'échelle de risque de récidive varie selon la race et le sexe.

Cette section comporte quatre étapes au cours desquelles nous allons :
1. Vérifier les données, mettre en œuvre quelques étapes de prétraitement et inspecter les données
2. Exécuter une courte analyse exploratoire du score de récidive COMPAS, notre principale variable d'intérêt
3. Reproduire le modèle de régression logistique de l'analyse de ProPublica et interpréter les estimations
4. Calculer la précision prédictive des étiquettes de score de risque
    


# Vérification des données

Vérifiez les premières lignes de données du référentiel d'analyse compas de ProPublica sur GitHub :
    


In [ ]:
df_compas = pd.read_csv(url)
print("Forme : ", df_compas.shape) # Forme = Shape
df_compas.head(5)


## Remarques sur les données

Reportez-vous à la description de la [méthodologie de collecte des données](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm). Les points importants sont soulignés ci-dessous ; consultez la description complète de ProPublica pour plus de détails.

> **Objectif :** Nous avons examiné plus de 10 000 accusés au criminel dans le comté de Broward, en Floride, et nous avons comparé leurs taux de récidive prédits au taux réellement observé sur une période de deux ans.
>
> **Données d'entrée de l'outil COMPAS (personnes concernées) :** Lorsque la plupart des accusés sont incarcérés, ils répondent à un questionnaire COMPAS. Leurs réponses sont saisies dans le logiciel COMPAS pour générer plusieurs scores, notamment des prédictions du risque de récidive et du risque de récidive violente.
>
> **Comment ProPublica a obtenu les données d'entrée de COMPAS :** Dans le cadre d'une demande d'accès aux documents publics, ProPublica a obtenu deux années de scores COMPAS auprès du bureau du shérif du comté de Broward en Floride. Nous avons reçu des données pour l'ensemble des 18 610 personnes qui ont obtenu un score en 2013 et 2014.
>
> **Sortie de l'outil COMPAS :** Chaque accusé en attente de procès a reçu au moins trois scores COMPAS : « Risque de récidive », « Risque de violence » et « Risque de non-comparution ». [...] Les scores COMPAS pour chaque accusé variaient de 1 à 10, dix étant le risque le plus élevé. Les scores de 1 à 4 ont été étiquetés par COMPAS comme « faibles » ; de 5 à 7 ont été étiquetés « moyens » ; et de 8 à 10 ont été étiquetés « élevés ».
>
> **Intégration des données (regroupement des enregistrements) :** À partir de la base de données des scores COMPAS, nous avons établi un profil des antécédents criminels de chaque personne, avant et après l'obtention du score. Nous avons recueilli des dossiers criminels publics sur le site Web du greffe du comté de Broward jusqu'au 1er avril 2016. En moyenne, les accusés de notre ensemble de données n'ont pas été incarcérés pendant 622,87 jours (écart type : 329,19). Nous avons apparié les dossiers criminels aux dossiers COMPAS en utilisant le prénom, le nom de famille et la date de naissance de la personne. Il s'agit de la même technique que celle utilisée dans l'étude de validation COMPAS du comté de Broward menée par des chercheurs de l'université d'État de Floride en 2010. Nous avons téléchargé environ 80 000 dossiers criminels à partir du site Web du greffe du comté de Broward.
>
> **Qu'est-ce que la recidive ?** Northpointe a défini la recidive comme « une arrestation avec prise d'empreintes digitales impliquant une accusation et un dépôt pour tout code de déclaration uniforme de la criminalité (DUC) ». Nous avons interprété cela comme signifiant une infraction criminelle ayant donné lieu à une mise en détention et ayant eu lieu après le crime pour lequel la personne a obtenu un score COMPAS. [...] Pour la plupart de nos analyses, nous avons défini la récidive comme une nouvelle arrestation dans un délai de deux ans.
    


# Inspecter les données

Par souci de commodité, voici un tableau des définitions des variables :

| Variable    | Description |
| ----------- | ----------- |
| age       |  Âge de l'accusé   |
| age_cat   |  Tranche d'âge. Il peut s'agir de < 25, 25-45, >45    |
| sex   |  Sexe de l'accusé. Il s'agit soit de « Male » (homme), soit de « Female » (femme)       |
| race   |  Race de l'accusé. Il peut s'agir de « African-American » (Afro-Américain), « Caucasian » (Blanc), « Hispanic » (Hispanique), « Asian » (Asiatique) ou « Other » (Autre)      |
| c_charge_degree   |   Inculpation. Soit « M » pour délit, « F » pour crime, soit « O » (n'entraînant pas de peine d'emprisonnement)    |
| priors_count   |   Nombre de crimes antérieurs commis par l'accusé      |
| days_b_screening_arrest   |  Jours entre l'arrestation et l'évaluation COMPAS       |
| decile_score   |  Le score COMPAS estimé par le système. Il se situe entre 0 et 10       |
| score_text   |  Score décile. Il peut être « Low » (faible) (1-4), « Medium » (moyen) (5-7) ou « High » (élevé) (8-10)       |
| is_recid   |  Indique si l'accusé a récidivé. Il peut être égal à 0, 1 ou -1      |
| two_year_recid   |  Indique si l'accusé a récidivé dans les deux ans suivant l'évaluation COMPAS      |
| c_jail_in   |   Date à laquelle l'accusé était en prison      |
| c_jail_out  |   Date à laquelle l'accusé a été libéré de prison     |

\
# **TODO 1** Tracer la distribution de l'âge, de la race et du sexe dans les données importées (```df_compas```) :
    



In [ ]:
#Votre travail ici :




In [ ]:

# @title Réponse
df_compas["age"].hist()
plt.xlabel("Âge")
plt.ylabel("Fréquence")
plt.show()

df_compas["race"].value_counts().plot(kind = "bar")
plt.xlabel("Race")
plt.ylabel("Fréquence")
plt.show()

df_compas["sex"].value_counts().plot(kind = "bar")
plt.xlabel("Sexe")
plt.ylabel("Fréquence")
plt.show()

# Préparer les données


ProPublica a mis en œuvre quelques étapes de prétraitement. Tout d'abord, ils ont généré un sous-ensemble des données avec quelques variables d'intérêt. Ici, nous sélectionnons encore moins de variables, en ne conservant que celles que nous utiliserons dans ce bloc-notes.


In [ ]:
cols_to_keep = ["id", "age", "c_charge_degree", "race", "age_cat", "score_text",
                "sex", "priors_count", "days_b_screening_arrest",
                "decile_score", "is_recid", "two_year_recid"]

# (df_selected)
df_selected = df_compas[cols_to_keep].copy()

print("Forme : ", df_selected.shape) # Forme
df_selected.head()



Prenez un moment pour vous familiariser avec les variables et la structure des données. ProPublica a filtré les données ci-dessus en supprimant les lignes où :

1.  Le score COMPAS est manquant.
2.  La date de l'infraction pour laquelle le score COMPAS du défendeur a été calculé n'était pas dans les 30 jours suivant la date de l'arrestation. ProPublica a supposé que l'infraction pourrait ne pas être correcte dans ces cas.
3.  L'indicateur de récidive est « -1 ». Dans de tels cas, ProPublica n'a trouvé aucun enregistrement COMPAS.
4.  L'accusation est « O ». Il s'agit d'infractions au code de la route ordinaires qui n'entraînent pas de peine d'emprisonnement.

Nous implémentons ces conditions ici :
    


In [ ]:
df_analysis = df_selected[
    (df_selected.score_text != "N/A") &
    (df_selected.days_b_screening_arrest <= 30) &
    (df_selected.days_b_screening_arrest >= -30) &
    (df_selected.is_recid != -1) &
    (df_selected.c_charge_degree != "O")
    ].copy()



Notez que ProPublica n'a inclus que les personnes ayant récidivé dans un délai de deux ans ou ayant passé au moins deux ans hors d'un établissement correctionnel. Cette étape de prétraitement est "intégrée" aux données que nous avons importées depuis GitHub dans ce bloc-notes.

# **TODO 2** Vérifiez les dimensions (c'est-à-dire le nombre de variables et d'observations) des données importées (```df_compas```) et prétraitées (```df_analysis```) :
    


In [ ]:
#Votre travail ici :




In [ ]:
#@title Réponse
print("Données importées", df_compas.shape)
print("Données après la sélection des variables", df_selected.shape)
print("Données après le filtrage des observations", df_analysis.shape)


Prenez l'étape supplémentaire de vous assurer que le score de décile (abordé ci-dessous) est numérique :

    


In [ ]:
df_analysis["decile_score"] = pd.to_numeric(df_analysis["decile_score"])


# Inspecter les données à nouveau


# **TODO 3** Inspectez à nouveau les variables importantes dans les données après les étapes de prétraitement. Tracez la distribution de l'âge, de l'origine ethnique et du sexe dans les données prétraitées (```df_analysis```) et comparez ces distributions aux données importées (```df_compas```) :


In [ ]:
#Votre travail ici :


In [ ]:
#@title Réponse
plt.hist(df_compas["age"], alpha = 0.5, label = "importé")
plt.hist(df_analysis["age"], alpha = 0.5, label= "filtré")
plt.legend(loc = "upper right")
plt.xlabel("Âge")
plt.ylabel("Fréquence")
plt.show()

df_compas["race"].value_counts().plot(kind = "bar", alpha=0.5)
df_analysis["race"].value_counts().plot(kind = "bar", alpha=0.5, color='orange')
plt.xlabel("Origine ethnique (race)") # race
plt.ylabel("Fréquence")
plt.show()

df_compas["sex"].value_counts().plot(kind = "bar", alpha=0.5)
df_analysis["sex"].value_counts().plot(kind = "bar", color='orange', alpha=0.5)
plt.xlabel("Sexe (sex)") # sex
plt.ylabel("Fréquence")
plt.show()




# **TODO 4** Observez que nous itérons à travers l'analyse de données : importation, inspection et profilage, prétraitement, et profilage à nouveau. Générez un tableau croisé résumant le nombre d'observations par origine ethnique (race) et sexe (sex):


In [ ]:
#Votre travail ici :


In [ ]:
#@title Réponse
df_analysis.pivot_table(values = ["id"], columns = ["race"],
                        index = "sex", aggfunc = lambda x: len(x))



# Analyse exploratoire


Concentrons-nous sur la variable principale qui nous intéresse : le score de récidive COMPAS. Dans cette analyse exploratoire, nous nous intéressons à la variable nommée « decile_score ».

L’analyse de ProPublica note : « Les juges sont souvent confrontés à deux ensembles de scores du système COMPAS : l’un qui classe les personnes en risque élevé, moyen ou faible, et un score décile correspondant. »

Tracer la distribution du score décile pour les hommes et pour les femmes. Dans quelle mesure ces distributions diffèrent-elles ?


In [ ]:
# tracer le score décile par sexe
df_female = df_analysis[(df_analysis.sex == "Female")].copy()
df_male   = df_analysis[(df_analysis.sex == "Male")].copy()

fig = plt.figure(figsize = (12, 6))
fig.add_subplot(121)

plt.hist(df_female["decile_score"], ec = "white",
         weights = np.ones(len(df_female["decile_score"])) /
         len(df_female["decile_score"]))
plt.xlabel("Score décile (0-10)")
plt.ylabel("Pourcentage de cas")
plt.title("Scores déciles des femmes défenderesses") # défendeur, accusé (Defendants)
plt.ylim([0, 0.25])

fig.add_subplot(122)
plt.hist(df_male["decile_score"], ec = "white",
         weights = np.ones(len(df_male["decile_score"])) /
         len(df_male["decile_score"]))
plt.xlabel("Score décile (0-10)")
plt.ylabel("Pourcentage de cas")
plt.title("Scores déciles des hommes défendeurs") # défendeur, accusé (Defendants)
plt.ylim([0, 0.25])

plt.show()



# **TODO 5** Qu'en est-il de l'origine ethnique ? Reproduisez les graphiques ci-dessus pour les défendeurs noirs et les défendeurs blancs : # (Black defendants, White defendants)
    



In [ ]:
#Votre travail ici :



In [ ]:
#@title Réponse

# Tracer le décile de score par race
df_black = df_analysis[(df_analysis.race == "African-American")]
df_white = df_analysis[(df_analysis.race == "Caucasian")]

fig = plt.figure(figsize = (12, 6))
fig.add_subplot(121)

plt.hist(df_black["decile_score"], ec = "white",
         weights = np.ones(len(df_black["decile_score"])) /
         len(df_black["decile_score"]))
plt.xlabel("Score Décile (0-10)")
plt.ylabel("Pourcentage de Cas")
plt.title("Scores Décile des Prévenus Noirs")
plt.ylim([0, 0.30])

fig.add_subplot(122)
plt.hist(df_white["decile_score"], ec = "white",
         weights = np.ones(len(df_white["decile_score"])) /
         len(df_white["decile_score"]))
plt.xlabel("Score Décile (0-10)")
plt.ylabel("Pourcentage de Cas")
plt.title("Scores Décile des Prévenus Blancs")
plt.ylim([0, 0.30])

plt.show()

# **TÂCHE 6** Résumez la différence entre la distribution des scores de décile pour les accusés noirs et les accusés blancs (trois phrases maximum) :


Votre travail ici :
    


# **TODO 7** Tracer la distribution des « étiquettes de risque » (la variable est nommée "score_text") assignées par COMPAS pour les défendeurs noirs et les défendeurs blancs :
    


In [ ]:
#Votre travail ici :


In [ ]:
#@title Réponse

# calcul du groupe à risque par race
fig = plt.figure(figsize = (12, 6))

fig.add_subplot(121)
(df_black["score_text"].value_counts().reindex(['Low', 'Medium', 'High']) /
    len(df_black)).plot(kind = "bar")
plt.xlabel("Score Label")
plt.ylabel("Pourcentage de cas")
plt.title("Répartition des scores pour les accusés noirs")
plt.ylim([0, .7])

fig.add_subplot(122)
(df_white["score_text"].value_counts().reindex(['Low', 'Medium', 'High']) /
    len(df_white)).plot(kind = "bar")
plt.xlabel("Score Label")
plt.ylabel("Pourcentage de cas")
plt.title("Répartition des scores pour les accusés blancs")
plt.ylim([0, .7])

plt.show()



# Biais dans COMPAS


ProPublica s'est concentré sur le biais racial dans l'algorithme COMPAS. En termes généraux, ProPublica a analysé (i) comment les *scores de risque* varient selon l'origine ethnique et (ii) dans quelle mesure les *étiquettes de risque* attribuées aux accusés correspondent à leur récidive observée et comment cela varie selon l'origine ethnique. Nous allons (approximativement) reproduire cette analyse ci-dessous.


## Préparer les données pour la régression logistique

ProPublica a utilisé un modèle de régression logistique pour analyser la variation des scores de risque en fonction de l'origine ethnique. Nous allons préparer les données en encodant les variables catégorielles en utilisant le one-hot encoding.
    


In [ ]:
print(df_analysis.dtypes)


In [ ]:
for i, col_type in enumerate(df_analysis.dtypes):
        if col_type == "object":
            print("\nLa variable {} prend les valeurs : {}".format( # Variable (col_type), valeurs (values)
                df_analysis.columns[i],
                df_analysis[df_analysis.columns[i]].unique()))


In [ ]:
df_logistic = df_analysis.copy()

# encodage one-hot
df_logistic = pd.get_dummies(df_logistic,
                             columns = ["c_charge_degree", "race",
                                        "age_cat", "sex"])

# transformer score_text en variable binaire où low = {low}
# et high = {medium, high}
df_logistic["score_binary"] = np.where(df_logistic["score_text"] != "Low",
                                       "High", "Low")
df_logistic["score_binary"] = df_logistic["score_binary"].astype('category')

# renommer les colonnes pour qu'elles soient plus instructives et cohérentes avec les statsmodel
# exigences relatives aux noms de variables
df_logistic.columns = df_logistic.columns.str.replace(' ', '_')
df_logistic.columns = df_logistic.columns.str.replace('-', '_')

#  (renamed_cols)
renamed_cols = {'age_cat_25___45':'age_cat_25_to_45',
                'c_charge_degree_F':'Felony', # Felony (F)
                'c_charge_degree_M':'Misdemeanor'} # Misdemeanor (M)

df_logistic = df_logistic.rename(columns = renamed_cols)


Vérifiez que le recodage a donné la structure de données souhaitée :
    


In [ ]:
df_logistic.head()


## Estimer un modèle de régression logistique

Suivant ProPublica, nous spécifions le modèle de régression logistique suivant :
    


In [ ]:
# Variables explicatives
    # explicatif (explanatory)
explanatory = "priors_count + two_year_recid + Misdemeanor + \
age_cat_Greater_than_45 + age_cat_Less_than_25 + \
race_African_American + race_Asian + race_Hispanic + race_Native_American + \
race_Other + sex_Female"

# Variable réponse
# (response)
response = "score_binary"

# Formule
# (formula)
formula = response + " ~ " + explanatory
print(formula)



Ajustons le modèle :


In [ ]:
# Remarque : utiliser family = sm.families.Binomial() spécifie une régression logistique
model = sm.formula.glm(formula = formula,
                       family = sm.families.Binomial(),
                       data = df_logistic).fit() # (df_logistic = données pour la régression logistique)

print(model.summary())



## Interpréter les estimations


Prenez le temps de lire attentivement le résumé du modèle.

Une façon d'interpréter les estimations consiste à calculer les rapports de cotes. Pour calculer les rapports de cotes, nous prenons l'exponentielle des coefficients. Par exemple, prendre l'exponentielle du coefficient pour sex_Female ($\beta_{female}$ = 0.2213) retournera la cote de score_text prenant la valeur "high" pour une femme par rapport à un homme.

# **TODO 8** Calculer ce rapport de cotes ici :


In [ ]:
#Votre travail ici :


In [ ]:
#@title Réponse
np.exp(0.2213)



En d'autres termes, la probabilité que COMPAS qualifie un accusé de « risque élevé » de récidive est 1,25 fois plus élevée pour une femme que pour un homme.

# **TODO 9** Calculer le rapport de cotes pour tous les coefficients du modèle :
    


In [ ]:
#Votre travail ici :




In [ ]:
#@title Réponse
np.exp(model.params)




Prenez le temps de lire ces coefficients. Quelle est la catégorie de référence pour chaque variable ? (par exemple, pour les femmes, la catégorie de référence est homme). Pensez en termes de comparaisons, par exemple :

> Une personne avec une valeur de [ &nbsp; &nbsp; ] sur la variable [ &nbsp; &nbsp; ] a [ &nbsp; &nbsp; ] fois plus de chances d'être étiquetée à haut risque par rapport à une personne avec une valeur de [ &nbsp; &nbsp; ] sur la variable [ &nbsp; &nbsp; ]

Dans l'exemple féminin ci-dessus, cela pourrait être dit :

> « Une personne avec une valeur de femme sur la variable sexe a 1,25 fois plus de chances d'être étiquetée à haut risque par rapport à une personne avec une valeur d'homme sur la variable sexe »

Bien sûr, nous devrions être plus directs lors de la rédaction des résultats. « Une personne avec une valeur d'homme sur la variable sexe » est plutôt verbeux; « hommes » suffira. Interpréter les estimations du modèle en termes simples est une compétence sous-estimée.


# **TODO 10** Résumez les probabilités associées à la variable "age_cat" (deux phrases maximum) :

    


Votre travail ici :
    


## Précision prédictive

En termes d'équité, ProPublica s'est concentré sur la précision prédictive de l'algorithme COMPAS. Dans ce cas, la précision prédictive fait référence à la concordance entre la récidive d'une personne et l'étiquette attribuée à cette personne par l'algorithme COMPAS. Par exemple, à quelle fréquence COMPAS a-t-il prédit qu'une personne présentait un « risque élevé » de récidive et que cette personne a effectivement récidivé dans un délai de deux ans ? Nous pouvons penser à cela en termes de tableau 2x2 :

|      | N'a pas récidivé | A récidivé   |
| :---        |    :----:   |          ---: |
| **Étiqueté à risque élevé**  | A       | B   |
| **Étiqueté à faible risque**   | C       | D      |

ProPublica a rapporté A et D pour les accusés noirs et les accusés blancs, séparément.

# **TODO 11** Quels sont les termes génériques pour A et D ? Pourquoi se concentrer sur A et D ?


Votre travail ici :


ProPublica a utilisé un ensemble de données quelque peu différent pour calculer la précision prédictive de COMPAS. Dans cette section, nous utiliserons les données ```df_logistic``` que nous avons prétraitées ci-dessus par souci de concision. Notez donc que les chiffres que nous calculons ci-dessous ne correspondent pas à ceux rapportés par ProPublica. Générons un tableau croisé de la variable dénotant la récidive dans les deux ans (```is_recid```) et de la variable de score binaire (```score_binary```) :


In [ ]:
print("Tous les défendeurs")
pd.crosstab(df_logistic["score_binary"], df_logistic["is_recid"])


# **À FAIRE 12** En vous basant sur ce tableau croisé, indiquez le nombre de vrais positifs, de faux positifs, de vrais négatifs et de faux négatifs :
    


In [ ]:
true_positive = 1817#@param {type:"number"}
false_positive = 934#@param {type:"number"}
true_negative = 2248#@param {type:"number"}
false_negative = 1173#@param {type:"number"}


Vous pouvez calculer le taux de faux positifs en prenant FP / (FP + TN), où FP est le nombre de faux positifs et TN est le nombre de vrais négatifs. Calculez le taux de faux positifs :


In [ ]:
#Votre travail ici :



In [ ]:
#@title Answer / titre Réponse
print("All defendants")
print("False positive rate",
false_positive / (false_positive + true_negative) * 100) # (taux de faux positifs)

# **TODO 13** Maintenant, calculez le taux de *faux négatifs* : (indice : remplacez les termes dans la formule du taux de faux positifs dans la cellule de texte précédente)


In [ ]:
#Votre travail ici :


In [ ]:
#@title Réponse
print("Tous les défendeurs")
print("Taux de faux négatifs",
      false_negative / (false_negative + true_positive) * 100)




# **TODO 14** Comment les taux de faux positifs et de faux négatifs varient-ils selon le sexe ? Générons un tableau croisé de "score_binary" et "is_recid" pour les femmes accusées et calculons les taux de faux positifs et de faux négatifs pour les femmes :
    


In [ ]:
#Votre travail ici :


In [ ]:
#@title Réponse
mask = df_logistic["sex_Female"] == 1
print(pd.crosstab(df_logistic.loc[mask, "score_binary"],
                  df_logistic.loc[mask, "is_recid"]))
print("Female defendants")

In [ ]:
#@title Réponse
tp = 256
fp = 220
tn = 520
fn = 179
print("False positive rate", fp / (fp + tn) * 100) # (fp: faux positifs, tn: vrais négatifs)
print("False negative rate", fn / (fn + tp) * 100) # (fn: faux négatifs, tp: vrais positifs)


# **TODO 15** Calculer maintenant les taux de faux positifs et de faux négatifs pour les accusés de sexe masculin :


In [ ]:
#Votre travail ici :



In [ ]:
#@title Réponse
mask = df_logistic["sex_Male"] == 1
print(pd.crosstab(df_logistic.loc[mask, "score_binary"],
                  df_logistic.loc[mask, "is_recid"]))




In [ ]:
#@title Réponse
print("Prévenus de sexe masculin") # Male defendants
tp = 1561
fp = 714
tn = 1728
fn = 994
print("Taux de faux positifs", fp / (fp + tn) * 100) # Taux de faux positifs (False positive rate)
print("Taux de faux négatifs", fn / (fn + tp) * 100) # Taux de faux négatifs (False negative rate)



# **TODO 16** Comment les taux de faux positifs et de faux négatifs varient-ils selon l'origine ethnique ? Calculez le taux de faux positifs et le taux de faux négatifs pour les accusés blancs :
    


In [ ]:
#Votre travail ici :


In [ ]:
#@title Réponse
mask = df_logistic["race_Caucasian"] == 1
print(pd.crosstab(df_logistic.loc[mask, "score_binary"],
                  df_logistic.loc[mask, "is_recid"]))

In [ ]:
#@title Réponse
print("Accusés blancs")
tp = 430
fp = 266
tn = 963
fn = 444
print("Taux de faux positifs", fp / (fp + tn) * 100)
print("Taux de faux négatifs", fn / (fn + tp) * 100)


# **TODO 17** Finalement, calculez le taux de faux positifs et le taux de faux négatifs pour les accusés noirs :
    


In [ ]:
#Votre travail ici :


In [ ]:
#@title Réponse
mask = df_logistic["race_African_American"] == 1
print(pd.crosstab(df_logistic.loc[mask, "score_binary"],
                  df_logistic.loc[mask, "is_recid"]))

In [ ]:
#@title Réponse
print("Accusés noirs")
tp = 1248
fp = 581
tn = 821
fn = 525
print("Taux de faux positifs", fp / (fp + tn) * 100) # (fp: faux positifs, tn: vrais négatifs)
print("Taux de faux négatifs", fn / (fn + tp) * 100) # (fn: faux négatifs, tp: vrais positifs)


# **Deuxième partie : Détection et atténuation des biais à l'aide de Fairlearn**


#Détection du biais à l'aide de Fairlearn


## Biais en ML

Un algorithme de machine learning tentera de trouver des modèles, ou des généralisations, dans l'ensemble de données d'entraînement à utiliser lorsqu'une prédiction pour une nouvelle instance est nécessaire. Par exemple, le modèle peut découvrir un modèle selon lequel une personne ayant un salaire supérieur à 40 000 $ et une dette en cours inférieure à 5 $ est très susceptible de rembourser un prêt.

Cependant, il arrive que les modèles trouvés et reproduits par un modèle ne soient pas souhaitables ou, pire encore, illégaux. Par exemple, un modèle de remboursement de prêt peut déterminer que l'âge joue un rôle important dans la prédiction du remboursement parce que l'ensemble de données d'entraînement s'est avéré avoir un meilleur remboursement pour une tranche d'âge par rapport à une autre. Cela soulève deux problèmes : 1) l'ensemble de données d'entraînement peut ne pas être représentatif de la population réelle des demandes de prêt pour toutes les tranches d'âge, et 2) même s'il est représentatif, il est illégal (à quelques exceptions près) de fonder les décisions de prêt sur l'âge d'un demandeur, que cela constitue ou non une base de prédiction précise d'après les données historiques.

Le scénario du prêt décrit un exemple intuitif de biais illégal. Cependant, tous les biais indésirables en matière de machine learning ne sont pas illégaux ; ils peuvent également exister de manière plus subtile. Par exemple, une société de prêt peut souhaiter un portefeuille diversifié de clients à tous les niveaux de revenu, et jugera donc indésirable de consentir davantage de prêts aux personnes à revenu élevé qu'aux personnes à faible revenu. Bien que cela ne soit ni illégal ni contraire à l'éthique, c'est indésirable pour la stratégie de l'entreprise.

## La boîte à outils `Fairlearn`

Fairlearn est une boîte à outils conçue pour aider à résoudre ce problème grâce à des métriques d'équité et des atténuateurs de biais. Les métriques d'équité peuvent être utilisées pour vérifier la présence de biais dans les workflows de machine learning. Les atténuateurs de biais peuvent être utilisés pour surmonter les biais dans le workflow afin de produire un résultat plus équitable.

Comme ces deux exemples l'illustrent, une boîte à outils de détection et/ou d'atténuation des biais doit être adaptée au biais particulier qui nous intéresse. Plus précisément, nous devons définir le ou les attributs, appelés attributs protégés (ou sensibles) d'intérêt : l'attribut dont nous essayons de détecter et d'atténuer l'asymétrie/le biais. Ce terme suggère que le concepteur du système doit être sensible à ces caractéristiques lorsqu'il évalue et atténue l'équité de groupe.

Plusieurs étapes du pipeline de machine learning sont susceptibles d'être biaisées. Une façon utile de classer ces étapes consiste, intuitivement, à les distinguer « avant », « pendant » et « après » l'entraînement d'un modèle. Ces étapes sont communément appelées *prétraitement*, *traitement en cours* et *post-traitement* (dans Fairlearn, les techniques de traitement en cours sont disponibles dans le package *reductions*).


## Fairlearn

Dans la partie 2, nous allons utiliser Fairlearn pour détecter et atténuer les biais dans un classificateur. Nous utiliserons les [fichiers ACS PUMS](https://www.census.gov/programs-surveys/acs/microdata.html), en particulier une fraction de l'ensemble de données ACS Income, et nous allons entraîner un classificateur pour prédire si un individu a un salaire supérieur à 50 000 $. L'attribut protégé sera le sexe de l'individu.

Dans la partie 2, nous allons :

1. Explorer les métriques d'équité possibles
2. Entraîner un classificateur de régression logistique et évaluer l'équité de ce classificateur
3. Entraîner d'autres classificateurs de régression logistique avec des interventions de prétraitement et réévaluer l'équité
4. Comparer les résultats obtenus en 2 et 3
    


#1. Charger les données, effectuer une analyse exploratoire et prétraiter les données
Ensuite, nous allons charger le jeu de données Folktables. Le jeu de données Folktables est tiré des données du recensement américain et est conçu pour résoudre quelques tâches de prédiction simples. L'échantillon que nous extrayons est constitué de données de 2018 en Californie. Les noms des colonnes sont décrits dans le tableau ci-dessous. Notez que certaines variables catégorielles ont été mappées à des valeurs entières, que nous conserverons telles quelles pour les analyses suivantes.

Pour plus d'informations sur ce jeu de données, veuillez consulter l'article suivant (notamment la page 18) : https://eaamo2021.eaamo.org/accepted/acceptednonarchival/EAMO21_paper_16.pdf


| Nom de la colonne | Caractéristique | Description/Remarques |
| --- | ----------- | --- |
| PINCP | Revenu total de la personne | (Cible) 1 si >= 50 000 $, 0 si inférieur |
| SEX | Sexe | (Attribut sensible) Homme=1, Femme=2 |
| RAC1P | Race | (Attribut sensible) Blanc=1, Noir=2, Autres races entre 3 et 9 |
| AGEP | Âge | Varie de 0 à 99 |
| COW | Classe de travailleur | Varie de 1 à 9, voir l'article pour la description |
| SCHL | Niveau d'éducation | Varie de 1 à 24, voir l'article pour la description |
| MAR | État matrimonial | Varie de 1 à 5, voir l'article pour la description |
| OCCP | Profession | Codes tirés de l'échantillon de microdonnées à usage public (PUMS) du recensement américain, voir l'article |
| POBP | Lieu de naissance | Codes tirés de l'échantillon de microdonnées à usage public (PUMS) du recensement américain, voir l'article |
| RELP | Lien de parenté | Lien de parenté de l'individu avec la personne qui a répondu au recenseur. Varie de 0 à 17, voir l'article pour la description |
| WKHP | Heures travaillées par semaine | Varie de 0 à 99, en moyenne sur l'année précédente |


In [ ]:
# Lire le jeu de données folktables
    # Lisons le jeu de données folktables
full_df, features_df, target_df, groups_df = ACSData().return_acs_data_scenario(scenario="ACSIncome", subsample=70000)

print(full_df.shape)
full_df.head()



In [ ]:
# Vérifier les valeurs manquantes et les types de données
full_df.info()



In [ ]:
# modifier les types de données des caractéristiques catégorielles
numerical_cols = ['AGEP','WKHP']
categorical_cols = ['COW','SCHL','MAR','OCCP','POBP','RELP','RAC1P','SEX']

for col in categorical_cols:
  full_df[col] = full_df[col].astype('int')
  full_df[col] = full_df[col].astype('str')

full_df.info()
full_df.head()

Ensuite, nous allons effectuer une analyse exploratoire basique des données en commençant par tracer les distributions de nos caractéristiques.


In [ ]:
# Tracer la distribution des colonnes catégorielles
fig, ax = plt.subplots(2,4,figsize=(12,8))
ax[0,0].barh(full_df['COW'].value_counts().index[::-1], full_df['COW'].value_counts()[::-1])
ax[0,0].set_title('COW')

ax[0,1].barh(full_df['SCHL'].value_counts().index[:10][::-1], full_df['SCHL'].value_counts()[:10][::-1])
ax[0,1].set_title('SCHL (top-10)')

ax[0,2].barh(full_df['MAR'].value_counts().index[::-1], full_df['MAR'].value_counts()[::-1])
ax[0,2].set_title('MAR')

ax[0,3].barh(full_df['OCCP'].value_counts().index[:10][::-1], full_df['OCCP'].value_counts()[:10][::-1])
ax[0,3].set_title('OCCP (top-10)')

ax[1,0].barh(full_df['POBP'].value_counts().index[:10][::-1], full_df['POBP'].value_counts()[:10][::-1])
ax[1,0].set_title('POBP (top-10)')

ax[1,1].barh(full_df['RELP'].value_counts().index[:10][::-1], full_df['RELP'].value_counts()[:10][::-1])
ax[1,1].set_title('RELP (top-10)')

ax[1,2].barh(full_df['SEX'].value_counts().index[::-1], full_df['SEX'].value_counts()[::-1])
ax[1,2].set_title('SEX')
labels = ('Female = 2', 'Male = 1')
ax[1,2].set_yticklabels(labels)

ax[1,3].barh(full_df['RAC1P'].value_counts().index[::-1], full_df['RAC1P'].value_counts()[::-1])
ax[1,3].set_title('RAC1P')

fig.tight_layout()

In [ ]:
# Tracé de la distribution des caractéristiques numériques
fig, ax = plt.subplots(1,2,figsize=(12,8))

#histogramme pour AGEP (âge)
num_of_bins_agep = 10
y_vals_agep, x_vals_agep, e_agep = ax[0].hist(full_df['AGEP'], bins=num_of_bins_agep, edgecolor='black')
ax[0].set_title("Histogramme de AGEP")
ax[0].set_xlabel("âge") # (age)
ax[0].set_ylabel("Pourcentage")
y_max_agep = round((max(y_vals_agep) / len(full_df)) + 0.02, 2)
ax[0].set_yticks(ticks=np.arange(0.0, y_max_agep * len(full_df), 0.01 * len(full_df)))
ax[0].set_ylim(ax[0].get_yticks()[0], ax[0].get_yticks()[-1])
ax[0].yaxis.set_major_formatter(ticker.PercentFormatter(xmax=len(full_df)))

#histogramme pour WKHP (Heures travaillées par semaine)
num_of_bins_wkhp = 10
y_vals_wkhp, x_vals_wkhp, e_wkhp = ax[1].hist(full_df['WKHP'], bins=num_of_bins_wkhp, edgecolor='black')
ax[1].set_title("Histogramme de WKHP")
ax[1].set_xlabel("heures travaillées par semaine") # heures travaillées par semaine
ax[1].set_ylabel("Pourcentage")
y_max_wkhp = round((max(y_vals_wkhp) / len(full_df)) + 0.05, 2)
ax[1].set_yticks(ticks=np.arange(0.0, y_max_wkhp * len(full_df), 0.05 * len(full_df)))
ax[1].set_ylim(ax[1].get_yticks()[0], ax[1].get_yticks()[-1])
ax[1].yaxis.set_major_formatter(ticker.PercentFormatter(xmax=len(full_df)))




Comme nous pouvons le constater, la distribution des caractéristiques de ce jeu de données n'a rien de très inhabituel. On note également que la proportion d'hommes et de femmes est plutôt équilibrée.

On peut également examiner les corrélations par paires entre les caractéristiques numériques et notre variable cible.
    


In [ ]:
# Tracé des corrélations par paires entre les caractéristiques numériques

sns.heatmap(full_df.corr(), mask=np.identity(len(full_df.corr())), annot=True, cmap='Blues')
plt.show()


Ici, nous pouvons voir qu'il n'y a que des corrélations relativement faibles entre notre variable cible (PINCP) et nos caractéristiques numériques d'âge et d'heures travaillées par semaine.


Ensuite, nous pouvons examiner la distribution de notre variable cible ainsi que la distribution conjointe de nos attributs protégés et cibles.


In [ ]:
# Examiner la distribution de la variable cible (target variable)
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
full_df['PINCP'].value_counts().divide(full_df.shape[0]).plot(kind='bar')
ax.set_xlabel('Revenu (Income)')
ax.set_ylabel('Fréquence (Frequency)')
plt.setp(ax.get_xticklabels(), rotation=0, ha='center')
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1))
labels_target = ('0 (<$50k)', '1 (>=$50k)')
ax.set_xticklabels(labels_target)
plt.show()


D'après le graphique ci-dessus, nous remarquons un déséquilibre considérable de la variable cible (target variable). Voyons à quoi ressemble cette distribution selon le sexe.


In [ ]:
#Afficher la distribution de la variable cible (target variable) chez les hommes et la distribution de la variable cible chez les femmes
hist_df = full_df.groupby(['SEX','PINCP']).size().to_frame('count').reset_index()
new_col = full_df.groupby(['SEX']).PINCP.value_counts(normalize=True).values
hist_df['frac'] = new_col
hist_df.replace({'SEX': {'1': 'Homme', '2': 'Femme'}}, inplace=True)
hist_df.replace({'PINCP': {0.0: '<$50k', 1.0: '>=$50k'}}, inplace=True)
sns.barplot(x='PINCP', y='frac', hue='SEX', data=hist_df)
plt.ylabel('Pourcentage du groupe')
plt.xlabel('Revenu (Income)')
plt.show()


In [ ]:
# Affichage du nombre d'hommes et de femmes par groupe de revenu (Income group)
full_df.groupby(['SEX', 'PINCP'])['PINCP'].count()



Ici, nous pouvons constater que la proportion d'hommes qui gagnent au moins $50 000 $ est plus élevée que la proportion de femmes qui gagnent au moins $50 000 $ dans cet ensemble de données. Ainsi, la constatation initiale concernant le déséquilibre de la variable cible est plus accentuée pour les femmes.


# **TODO 1**: Étant donné les graphiques ci-dessus, à quels résultats pouvons-nous nous attendre de la part de notre classificateur lorsqu'il s'agira d'étiqueter les hommes et les femmes comme ayant un revenu élevé ou faible ?


Votre réponse ici :


In [ ]:
# @title Réponse
'''Étant donné que la proportion d'hommes dans ce jeu de données qui gagnent au moins 50 000 $ est plus élevée que la proportion de femmes dans ce jeu de données
qui gagnent au moins 50 000 $, nous pourrions nous attendre à ce que notre classificateur ait un biais pour étiqueter les hommes comme ayant un revenu plus élevé que les femmes.'''



<!-- **Réponse :** Étant donné que la proportion d'hommes dans ce jeu de données qui gagnent au moins 50 000 $ est plus élevée que la proportion de femmes dans ce jeu de données
qui gagnent au moins 50 000 $, nous pourrions nous attendre à ce que notre classificateur ait un biais pour étiqueter les hommes comme ayant un revenu plus élevé que les femmes. -->


# **TODO 2** : Pourquoi ces données pourraient-elles être biaisées ? De quel type de biais s'agit-il ?

<!-- **Réponse :** Les hommes pourraient être plus enclins que les femmes à gonfler les déclarations de leurs revenus réels. De plus, il existe des disparités historiques documentées dans la rémunération entre les hommes et les femmes pour un travail similaire pour diverses raisons (disparités dans l'éducation, le marché du travail, etc.). Par conséquent, les différences de revenus dans cet ensemble de données pourraient être le reflet d'un écart salarial réel. Ces deux exemples seraient des exemples de « biais préexistants » dans cet ensemble de données. -->


Votre travail ici :


In [ ]:
# @title Réponse
'''Les hommes pourraient être plus enclins que les femmes à gonfler les rapports sur leurs revenus réels. De plus, il existe des écarts de rémunération historiques et documentés entre les hommes et les femmes pour un travail similaire, et ce, pour diverses raisons (écarts de niveau de scolarité, marché du travail, etc.). Par conséquent, les différences de revenus dans cet ensemble de données pourraient refléter un écart de rémunération réel. Ces deux exemples illustrent les « biais préexistants » dans cet ensemble de données.'''


#**TODO 3**: Écrivez un code qui reproduit l'histogramme ci-dessus pour les personnes noires et blanches dans les données.


In [ ]:
#Votre travail ici :




In [ ]:
# @title Réponse
hist_df = full_df.groupby(['RAC1P','PINCP']).size().to_frame('count').reset_index()
new_col = full_df.groupby(['RAC1P']).PINCP.value_counts(normalize=True).values
hist_df['frac'] = new_col
# (RAC1P: Race)
hist_df['RAC1P'] = hist_df['RAC1P'].map({'1': 'Blanc', '2': 'Noir'}).fillna('Autre')
# (PINCP: Revenu (Income))
hist_df.replace({'PINCP': {0.0: '<$50k', 1.0: '>=$50k'}}, inplace=True)
sns.barplot(x='PINCP', y='frac', hue='RAC1P', data=hist_df)
plt.ylabel('Pourcentage par groupe')
plt.xlabel('Revenu')
plt.show()


## Prétraitement
Ensuite, nous allons effectuer un prétraitement sur nos données afin de les préparer à être utilisées dans notre modèle.


In [ ]:
# standardiser les variables numériques
scaler = StandardScaler()
full_df[numerical_cols] = scaler.fit_transform(full_df[numerical_cols])
display(full_df)




In [ ]:
# Encodage one-hot des variables catégorielles
full_df = pd.get_dummies(full_df, columns=categorical_cols)
display(full_df)



In [ ]:
# Puisque l'attribut sexe est déjà binaire, on peut supprimer une des colonnes redondantes
#note: les hommes sont maintenant étiquetés 1 et les femmes sont étiquetés 0
full_df.drop(columns=['SEX_2'], inplace=True)
full_df.rename(columns={'SEX_1':'SEX'}, inplace=True)

full_df.head()



#2. Entraîner un classificateur de régression logistique
Ensuite, nous allons diviser nos données en ensembles d'entraînement et de test de manière aléatoire. Puis, nous allons entraîner un classificateur de régression logistique et évaluer les biais possibles au sein de ce classificateur.

### Aparté : La précision comme métrique

Le ML traditionnel (c'est-à-dire sans accent sur l'équité) mesure souvent la qualité d'un classificateur par sa **précision**, ou la fraction d'échantillons qui ont été étiquetés correctement :
$$
\text{Précision} = \frac{\text{Nombre de personnes étiquetées correctement}}{\text{Nombre total de personnes}}
$$
Ceci peut également être exprimé en utilisant les termes de la ["matrice de confusion"](https://fr.wikipedia.org/wiki/Matrice_de_confusion), où nous avons
- $\text{TP} = $ "Vrais positifs" $ = \text{Nombre de personnes correctement étiquetées comme positives}$
- $\text{FP} = $ "Faux positifs" $ = \text{Nombre de personnes incorrectement étiquetées comme positives}$
- $\text{TN} = $ "Vrais négatifs" $ = \text{Nombre de personnes correctement étiquetées comme négatives}$
- $\text{FN} = $ "Faux négatifs" $ = \text{Nombre de personnes incorrectement étiquetées comme négatives}$

Ce qui nous permet d'exprimer la précision comme suit :
$$
\text{Précision} = \frac{TP + TN}{TP + FP + TN + FN}
$$
    


In [ ]:
# Diviser les données en ensembles d'entraînement et de test
target = full_df['PINCP']
full_df.drop(columns='PINCP', inplace=True)

#note: ici, nous définissons une valeur pour le paramètre random_state (graine) afin que les résultats de ce laboratoire restent cohérents
X_train, X_test, y_train, y_test = train_test_split(full_df, target, test_size=0.2, random_state=4)

# (X_train, X_test : données d'entraînement et de test, y_train, y_test : étiquettes correspondantes)
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')



#**TODO 4**: Quelle serait la précision ($\frac{(VP + VN)}{(VP + FP + VN + FN)}$) d'un classificateur qui prédit toujours la classe majoritaire (classificateur de base) ?
Étant donné que la classe majoritaire est 0, la précision de ce classificateur de classe majoritaire serait le nombre de 0 sur le nombre total d'enregistrements.


In [ ]:
# Votre réponse ici :



In [ ]:
# @title Réponse
# Donner aux étudiants ce qui suit :
# y_test.value_counts()
# # print(f'General baseline accuracy: {baseline_accuracy:.4f}')

target_zero = y_test.value_counts()[0] # target_zero : nombre d'éléments avec target = 0
target_one = y_test.value_counts()[1] # target_one : nombre d'éléments avec target = 1
baseline_accuracy = target_zero / (target_zero+target_one)

print(f'General baseline accuracy: {baseline_accuracy:.4f}')


#**TODO 5**: Quelle serait l'accuracy ($\frac{(TP + TN)}{(TP + FP + TN + FN)}$) pour les groupes Homme et Femme en considérant un classifieur qui prédit toujours la classe majoritaire (classifieur de base) pour chacun de ces groupes ?


In [ ]:
# Your work here:


In [ ]:
# @title Réponse
# # print(f'Précision de base chez les hommes (Male baseline accuracy): {male_baseline_accuracy:.4f}')
# # print(f'Précision de base chez les femmes (Female baseline accuracy): {female_baseline_accuracy:.4f}')

# On récupère le nombre d'hommes avec 'Outcome' (y_test) = 0 (male_zero) et = 1 (male_one)
male_zero = y_test[X_test['SEX']==1].value_counts()[0]
male_one = y_test[X_test['SEX']==1].value_counts()[1]
male_baseline_accuracy = male_zero / (male_zero + male_one)
print(f'Male baseline accuracy: {male_baseline_accuracy:.4f}')

# On récupère le nombre de femmes avec 'Outcome' (y_test) = 0 (female_zero) et = 1 (female_one)
female_zero = y_test[X_test['SEX']==0].value_counts()[0]
female_one = y_test[X_test['SEX']==0].value_counts()[1]
female_baseline_accuracy = female_zero / (female_zero + female_one)
print(f'Female baseline accuracy: {female_baseline_accuracy:.4f}')




In [ ]:
# Implémentation de la régression logistique
clf = LogisticRegression()
clf.fit(X_train, y_train)
clf_accuracy = clf.score(X_test, y_test)

clear_output()

print(f'Logistic Regression test accuracy: {clf_accuracy:.4f}')


## Évaluer l'équité


## Évaluer l'équité

Ensuite, nous allons évaluer l'équité de notre classificateur sur l'ensemble de test. Nous définissons d'abord le **taux de sélection** du classificateur sur un groupe :
$$
\text{Taux de sélection} = \frac{\text{Nombre de personnes classées positives}}{\text{Nombre total de personnes}}
$$
Nous allons calculer le taux de sélection chez les hommes et chez les femmes, et les comparer. La différence entre leurs taux de sélection est appelée **différence de parité démographique**, et le ratio de leurs taux est appelé **ratio de parité démographique**.

En général, si nous avons plus de 2 classes,
- La différence de parité démographique est la différence entre le taux de sélection le plus élevé et le taux de sélection le plus faible, elle est donc toujours positive. Une différence de parité démographique de 0 signifie que tous les groupes ont le même taux de sélection.

- Le ratio de parité démographique est le ratio du plus petit au plus grand taux de sélection, il est donc toujours compris entre 0 et 1, où un ratio de 1 signifie que tous les groupes ont le même taux de sélection.

<!-- Next, we will evaluate the fairness of our classifier on the test set.  We will first focus on two metrics - demographic parity difference and demographic parity ratio.  **Demographic parity difference** is defined as the difference between the largest and the smallest group-level selection rate across all values of the sensitive feature(s).  A demographic parity difference of 0 means that all groups have the same selection rate.  **Demographic parity ratio** is defined as the ratio between the smallest and the largest group-level selection rate across all values of the sensitive feature(s).  A demographic parity ratio of 1 means that all groups have the same selection rate. -->

Plus formellement : soit $X$ un vecteur de caractéristiques utilisé pour les prédictions, $A$ une seule caractéristique sensible (comme l'âge ou l'origine ethnique), $Y$ le vrai label, et $h$ un classificateur ou prédicteur résultant d'un algorithme de ML. Alors:

*Différence de parité démographique* est définie comme $(max_a\mathbb{E}[h(X)~|~  A = a])~ - ~ (min_a\mathbb{E}[h(X)~|~  A = a]) $


*Ratio de parité démographique* est défini comme $\frac{max_a\mathbb{E}[h(X)~|~  A = a]}{min_a\mathbb{E}[h(X)~|~  A = a]} $
    


In [ ]:
# Évaluer l'équité du classificateur à l'aide de `demographic_parity_difference` et `demographic_parity_ratio`
#note: nous menons cette analyse sur l'ensemble de test

#calculer les prédictions du test
y_pred = clf.predict(X_test)

#calculer la différence de parité démographique et le ratio de parité démographique
# (demo_parity_diff, demo_parity_ratio)
demo_parity_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=X_test['SEX'])
demo_parity_ratio = demographic_parity_ratio(y_test, y_pred, sensitive_features=X_test['SEX'])

print(f'Différence de parité démographique: {demo_parity_diff:.4f}')
print(f'Ratio de parité démographique: {demo_parity_ratio:.4f}')

#calculer le taux de sélection pour les hommes et les femmes
#(male_selection_rate, female_selection_rate)
male_selection_rate = selection_rate(y_test[X_test['SEX']==1], y_pred[X_test['SEX']==1])
female_selection_rate = selection_rate(y_test[X_test['SEX']==0], y_pred[X_test['SEX']==0])

print(f'Taux de sélection des hommes: {male_selection_rate:.4f}')
print(f'Taux de sélection des femmes: {female_selection_rate:.4f}')



Ici, nous pouvons constater qu'il existe des différences substantielles dans les taux de sélection entre les hommes et les femmes, les hommes étant beaucoup plus susceptibles d'être classés dans la catégorie des revenus élevés.


Fairlearn fournit également la classe **fairlearn.metrics.MetricFrame** pour évaluer les disparités de traitement entre différentes sous-populations.

L'objet **fairlearn.metrics.MetricFrame** nécessite au minimum quatre arguments :

*   La ou les fonctions de métrique sous-jacentes à évaluer
*   Les vraies valeurs (true values)
*   Les valeurs prédites (predicted values)
*   Les valeurs des caractéristiques sensibles (sensitive feature values)

Les fonctions de métrique doivent avoir une signature ''fn(y_true, y_pred)'', c'est-à-dire ne nécessiter que deux arguments. Nous allons à nouveau examiner le taux de sélection, mais nous allons également examiner quelques autres métriques.  Nous utiliserons la précision (accuracy), le taux de sélection (selection rate), le taux de faux négatifs (false negative rate) et le taux de faux positifs (false positive rate).


In [ ]:
# Évaluer l'équité du classificateur en utilisant la classe MetricFrame pour la variable `SEX`

#changer les entrées de caractéristiques sensibles pour qu'elles soient 'male' et 'female' au lieu de 1 et 0
#(sensitive_feature_sex) variable sensible sexe
sensitive_feature_sex = X_test['SEX'].replace({0:'female', 1:'male'})

#métriques d'évaluation
#(metrics) métriques
metrics = {'accuracy': skm.accuracy_score,
           'selection_rate': selection_rate,  # i.e., the percentage of the population which have ‘1’ as their predicted label
           'FNR': false_negative_rate,
           'FPR': false_positive_rate
           }

#(grouped_on_sex) groupé par sexe
grouped_on_sex = MetricFrame(metrics=metrics,
                             y_true=y_test,
                             y_pred=y_pred,
                             sensitive_features=sensitive_feature_sex)



La classe **fairlearn.metrics.MetricFrame** a la propriété **overall**, qui évalue les métriques sur l'ensemble du jeu de données.


In [ ]:
grouped_on_sex.overall

# **TODO 6**: Évaluer l'équité pour la variable `RAC1P` (pour les individus noirs et blancs au minimum), et afficher `grouped_on_race.overall`


In [ ]:
# Votre travail ici :


In [ ]:
#@title Réponse
sensitive_feature_race = (X_test['RAC1P_1'] + 2 * X_test['RAC1P_2'] ).replace({0:'autre', 1:'blanc', 2:'noir'})
indices = sensitive_feature_race != 'autre'
#métriques d'évaluation
metrics = {'accuracy': skm.accuracy_score,
           'selection_rate': selection_rate,  # c'est-à-dire le pourcentage de la population dont l'étiquette prédite est « 1 »
           'FNR': false_negative_rate,
           'FPR': false_positive_rate
           }

# (race)
grouped_on_race = MetricFrame(metrics=metrics,
                             y_true=y_test[indices],
                             y_pred=y_pred[indices],
                             sensitive_features=sensitive_feature_race[indices])



In [ ]:
print(y_pred[indices].shape)




In [ ]:
grouped_on_race.overall


L'objet **fairlearn.metrics.MetricFrame** possède également la fonctionnalité **by_group**. Celle-ci affiche les métriques sélectionnées évaluées sur chaque sous-groupe défini par les catégories dans les sensitive_features (le sexe dans notre cas).


In [ ]:
grouped_on_sex.by_group

# **TODO 7**: Afficher également pour `RAC1P`


In [ ]:
# Votre travail ici :


In [ ]:
#@title Réponse
grouped_on_race.by_group




Reminder: Females are labeled 0 and males are labeled 1.

In [ ]:
# Traçons les valeurs des métriques (metrics)

metrics_1 = {'accuracy': skm.accuracy_score,
           'selection_rate': selection_rate,  # c'est-à-dire le pourcentage de la population dont l'étiquette prédite est "1".
           }

metrics_2 = {
           'FNR': false_negative_rate,
           'FPR': false_positive_rate
           }

grouped_on_sex_accuracy_selection = MetricFrame(metrics=metrics_1,
                             y_true=y_test,
                             y_pred=y_pred,
                             sensitive_features=sensitive_feature_sex)

grouped_on_sex_fpr_fnr = MetricFrame(metrics=metrics_2,
                             y_true=y_test,
                             y_pred=y_pred,
                             sensitive_features=sensitive_feature_sex)

grouped_on_sex_accuracy_selection.by_group.plot.bar(
    subplots=False,
    figsize=(10, 7),
    ylim=[0,1],
    title="Précision (accuracy) et taux de sélection (selection rate) par sexe",
    )

grouped_on_sex_fpr_fnr.by_group.plot.bar(
    subplots=False,
    figsize=(10, 7),
    ylim=[0,1],
    title="Taux de faux négatifs (FNR) et taux de faux positifs (FPR) par sexe",
    )




#**À FAIRE 8** : Reproduisez les graphiques ci-dessus pour `RAC1P` et au moins pour les personnes noires/blanches dans les données.


In [ ]:
# Votre réponse ici :



In [ ]:
# @title Réponse
grouped_on_race_accuracy_selection = MetricFrame(metrics=metrics_1,
                             y_true=y_test,
                             y_pred=y_pred,
                             sensitive_features=sensitive_feature_race)

grouped_on_race_fpr_fnr = MetricFrame(metrics=metrics_2,
                             y_true=y_test,
                             y_pred=y_pred,
                             sensitive_features=sensitive_feature_race)

grouped_on_race_accuracy_selection.by_group.plot.bar(
    subplots=False,
    figsize=(10, 7),
    ylim=[0,1],
    title="Précision et taux de sélection par origine ethnique", # précision, taux de sélection (accuracy, selection rate)
    )

grouped_on_race_fpr_fnr.by_group.plot.bar(
    subplots=False,
    figsize=(10, 7),
    ylim=[0,1],
    title="Taux de faux négatifs (FNR) et taux de faux positifs (FPR) par origine ethnique", # FNR, FPR
    )




# **À FAIRE 9**: Observez que la précision pour les groupes hommes et femmes est comparable, mais que nous constatons des disparités dans les taux de FPR et de FNR.  Quel groupe bénéficie des écarts dans les FPR et FNR indiqués ci-dessus ? Si vous déployiez ce système, comment mesureriez-vous les performances (par exemple, la précision, le FNR, le FPR) ? (Rappel : les femmes sont étiquetées 0, les hommes sont étiquetés 1)

    


Votre réponse ici :


In [ ]:
#@title Réponse
'''
<Les hommes bénéficient à la fois d'un taux de faux négatifs plus faible et d'un taux de faux positifs plus élevé. Ils sont moins susceptibles d'être classés à tort
comme ayant un « faible revenu » et sont également plus susceptibles d'être classés à tort comme ayant un « revenu élevé ». Lors de la conception d'un classificateur, nous voudrions regarder
au-delà de la précision et du taux de sélection et tenir compte des FPR et FNR.'''




#3. Entraîner un classificateur de régression logistique « aveugle » (équité par l'ignorance)


Ensuite, nous allons supprimer l'attribut protégé "sex" de nos données et voir quel effet cela a sur les performances de notre classificateur.


In [ ]:
#Supprimer l'attribut sensible des données
X_train_blind = X_train.drop(columns='SEX')
X_test_blind = X_test.drop(columns='SEX')



In [ ]:
# Implémenter la régression logistique
clf_blind = LogisticRegression()
clf_blind.fit(X_train_blind, y_train)
clf_blind_accuracy = clf_blind.score(X_test_blind, y_test)

clear_output()

print(f'Exactitude du test de régression logistique (sans attribut sensible) : {clf_blind_accuracy:.4f}')


In [ ]:
# Rappelons l'équité de l'exactitude du classificateur d'origine
print('Rappelons le classificateur d\'origine :')
print(f'Exactitude du test de régression logistique : {clf_accuracy:.4f}')
print(f'Différence de parité démographique : {demo_parity_diff:.4f}')
print(f'Ratio de parité démographique : {demo_parity_ratio:.4f}')
print(f'Taux de sélection des hommes : {male_selection_rate:.4f}')
print(f'Taux de sélection des femmes : {female_selection_rate:.4f}')


In [ ]:
# Évaluer l'équité du classificateur aveugle

#calculer les prédictions du test
y_pred_blind = clf_blind.predict(X_test_blind)

#calculer la différence de parité démographique et le ratio de parité démographique
# (demo_parity_diff_blind, demo_parity_ratio_blind)
demo_parity_diff_blind = demographic_parity_difference(y_test, y_pred_blind, sensitive_features=X_test['SEX'])
demo_parity_ratio_blind = demographic_parity_ratio(y_test, y_pred_blind, sensitive_features=X_test['SEX'])

print(f'Différence de parité démographique (sans attribut sensible) : {demo_parity_diff_blind:.4f}')
print(f'Ratio de parité démographique (sans attribut sensible) : {demo_parity_ratio_blind:.4f}')

#calculer le taux de sélection pour les hommes et les femmes
# (male_selection_rate_blind, female_selection_rate_blind)
male_selection_rate_blind = selection_rate(y_test[X_test['SEX']==1], y_pred_blind[X_test['SEX']==1])
female_selection_rate_blind = selection_rate(y_test[X_test['SEX']==0], y_pred_blind[X_test['SEX']==0])

print(f'Taux de sélection des hommes (sans attribut sensible) : {male_selection_rate_blind:.4f}')
print(f'Taux de sélection des femmes (sans attribut sensible) : {female_selection_rate_blind:.4f}')


# **TODO 10:** Décrivez les différences entre les deux modèles en termes de précision et d'équité entre les groupes hommes et femmes ?


Votre réponse ici :

In [ ]:
'''Ici, nous pouvons constater que la suppression de l'attribut protégé de nos données a eu un impact minime sur la précision du modèle. En revanche,
les indicateurs d'équité se sont considérablement améliorés à mesure que la différence de parité démographique a diminué et que le ratio de parité démographique a augmenté.'''




**Néanmoins**, nous constatons que la suppression de la caractéristique protégée n'a pas éliminé les biais au sein de notre classificateur, comme illustré ci-dessous.


In [ ]:
# Évaluer les biais du classificateur à l'aide de la classe MetricFrame
#(blind = sans caractéristique protégée)
grouped_on_sex_blind = MetricFrame(metrics=metrics,
                                     y_true=y_test,
                                     y_pred=y_pred_blind,
                                     sensitive_features=sensitive_feature_sex)




In [ ]:
grouped_on_sex_blind.overall



In [ ]:
grouped_on_sex_blind.by_group

## Affichage de toutes les métriques pour les données complètes et les données aveugles.


In [ ]:
#Comparaison des résultats : données complètes vs. données aveugles

#différence de parité démographique
# (demo_parity_diff, demo_parity_diff_blind)
demo_parity_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=X_test['SEX'])
demo_parity_diff_blind = demographic_parity_difference(y_test, y_pred_blind, sensitive_features=X_test['SEX'])

#ratio de parité démographique
# (demo_parity_ratio, demo_parity_ratio_blind)
demo_parity_ratio = demographic_parity_ratio(y_test, y_pred, sensitive_features=X_test['SEX'])
demo_parity_ratio_blind = demographic_parity_ratio(y_test, y_pred_blind, sensitive_features=X_test['SEX'])

#taux de sélection
# (male_selection_rate, male_selection_rate_blind)
male_selection_rate = selection_rate(y_test[X_test['SEX']==1], y_pred[X_test['SEX']==1])
male_selection_rate_blind = selection_rate(y_test[X_test['SEX']==1], y_pred_blind[X_test['SEX']==1])

# (female_selection_rate, female_selection_rate_blind)
female_selection_rate = selection_rate(y_test[X_test['SEX']==0], y_pred[X_test['SEX']==0])
female_selection_rate_blind = selection_rate(y_test[X_test['SEX']==0], y_pred_blind[X_test['SEX']==0])

#fnr (taux de faux négatifs)
# (male_fnr, male_fnr_blind)
male_fnr = false_negative_rate(y_test[X_test['SEX']==1], y_pred[X_test['SEX']==1])
male_fnr_blind = false_negative_rate(y_test[X_test['SEX']==1], y_pred_blind[X_test['SEX']==1])

# (female_fnr, female_fnr_blind)
female_fnr = false_negative_rate(y_test[X_test['SEX']==0], y_pred[X_test['SEX']==0])
female_fnr_blind = false_negative_rate(y_test[X_test['SEX']==0], y_pred_blind[X_test['SEX']==0])

#fpr (taux de faux positifs)
# (male_fpr, male_fpr_blind)
male_fpr = false_positive_rate(y_test[X_test['SEX']==1], y_pred[X_test['SEX']==1])
male_fpr_blind = false_positive_rate(y_test[X_test['SEX']==1], y_pred_blind[X_test['SEX']==1])

# (female_fpr, female_fpr_blind)
female_fpr = false_positive_rate(y_test[X_test['SEX']==0], y_pred[X_test['SEX']==0])
female_fpr_blind = false_positive_rate(y_test[X_test['SEX']==0], y_pred_blind[X_test['SEX']==0])

#graphique
labels = ['Diff. Parité Démo.','Ratio Parité Démo.','Taux Sélection (Homme)',
          'Taux Sélection (Femme)', 'FNR (Homme)', 'FNR (Femme)', 'FPR (Homme)',
          'FPR (Femme)']

Y_full = [demo_parity_diff, demo_parity_ratio, male_selection_rate,
          female_selection_rate, male_fnr, female_fnr, male_fpr, female_fpr]

Y_blind = [demo_parity_diff_blind, demo_parity_ratio_blind,
           male_selection_rate_blind, female_selection_rate_blind, male_fnr_blind,
           female_fnr_blind, male_fpr_blind, female_fpr_blind]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots(figsize=(15, 7))
rects1 = ax.bar(x - width/2, Y_full, width, label='Modèle Complet')
rects2 = ax.bar(x + width/2, Y_blind, width, label='Modèle Aveugle')

ax.set_title('Comparaison des Métriques', size=20)
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend(fontsize='x-large')
ax.bar_label(rects1, padding=3, fmt='%.3f')
ax.bar_label(rects2, padding=3, fmt='%.3f')
ax.set_ylim([0, 1])
fig.tight_layout()
plt.rcParams["figure.figsize"] = (18,8)
plt.show()


## Conclusion
**Résumé :**

Biais dans les systèmes d'IA : Le laboratoire a commencé par examiner les biais raciaux et sexistes de l'outil COMPAS dans les prédictions de récidive, illustrant la nécessité de l'équité dans l'IA.

Reproduction de l'analyse : Les participants ont reproduit les conclusions de ProPublica en utilisant l'ensemble de données COMPAS, en se concentrant sur la régression logistique pour détecter les schémas de biais.

Équité avec Fairlearn : Le laboratoire a présenté Fairlearn pour évaluer et atténuer les biais dans les modèles de Machine Learning, en utilisant l'ensemble de données ACS pour aborder l'équité dans les prédictions liées au sexe.

Atténuer les biais : Des étapes pratiques pour améliorer l'équité des modèles ont été abordées, notamment la modélisation sensible à l'équité et les ajustements pour équilibrer performance et équité.

IA responsable : Le laboratoire souligne l'importance de lutter contre les biais dans les systèmes d'IA et fournit des outils et des idées pour mettre en œuvre des pratiques d'IA responsables.


## Feedback

Merci de nous fournir vos commentaires afin que nous puissions améliorer nos travaux pratiques à l'avenir.


In [ ]:
# @title Générer un formulaire de commentaires (Exécuter la cellule)
from IPython.display import HTML

HTML(
    """
<iframe
	src="https://forms.gle/WUpRupqfhFtbLXtN6",
  width="80%"
	height="1200px" >
	Loading...
</iframe>
"""
)




<img src="https://baobab.deeplearningindaba.com/static/media/indaba-logo-dark.d5a6196d.png" width="50%" />
    
